In [3]:
import pandas as pd
import numpy as np 
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
import glob
import matplotlib.pyplot as plt

In [4]:
data_dict = {
    "alaa" : "team_images/Alaa/*", 
    "aya amr" : "team_images/Aya Amr/*",
    "aya sameh" : "team_images/Aya Sameh/*",
    "ehab" : "team_images/Ehab/*",
    "salman" : "team_images/Salman/*"
}

In [5]:
def face_detection(img):
    face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )
    # The detectMultiScale() method is used to identify faces of different sizes in the input image.
    face = face_classifier.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(200, 200),flags=cv2.CASCADE_SCALE_IMAGE)
# finally, the minSize parameter sets the minimum size of the object to be detected.
# The model will ignore faces that are smaller than the minimum size specified.
    return face

In [6]:
def extract_features(pixel_values,folder_path, label):
    file_names = glob.glob(folder_path)
    for file in file_names:
        gray_img = cv2.imread(file,0)
        face = face_detection(gray_img)
        for (x, y, w, h) in face:
            face_img = gray_img[y:y+h, x:x+w]
            face_img = cv2.resize(face_img, (100,100))
            gray_pixels = face_img.flatten().tolist()
            gray_pixels.append(label)
        
        pixel_values.append(gray_pixels)
        

In [7]:
pixel_values = []
for key in data_dict.keys():    
    extract_features(pixel_values, data_dict[key], key)

df = pd.DataFrame(pixel_values)

In [8]:
df.rename(columns={10000:"label"}, inplace=True)

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,label
0,158,214,212,221,213,217,209,199,206,202,...,153,158,153,160,181,190,187,178,185,alaa
1,112,185,150,206,212,218,217,213,213,201,...,119,116,139,162,162,169,168,170,176,alaa
2,207,218,221,223,218,210,206,206,200,192,...,160,178,184,183,190,161,136,127,74,alaa
3,157,221,213,222,216,215,205,200,201,201,...,154,159,173,193,192,182,183,177,50,alaa
4,216,217,219,212,204,202,208,213,187,187,...,149,164,172,178,182,190,182,131,115,alaa


In [10]:
def pca(data, n_components):
  """
  Performs principal component analysis on the data and returns the eigenvectors, eigenvalues, and transformed data.

  Args:
    data: The data to be analyzed.
    n_components: The number of principal components to return.

  Returns:
    eigenvectors: The eigenvectors of the principal components.
    eigenvalues: The eigenvalues of the principal components.
    transformed_data: The data transformed into the principal component space.
  """
  # Check if the data is a NumPy array or Pandas DataFrame.
  if isinstance(data, np.ndarray):
    pass
  elif isinstance(data, pd.DataFrame):
    data = data.to_numpy()
  
  # Center the data.
  mean = np.mean(data, axis=0)
  std = np.std(data, axis=0)
  data = (data - mean) / std

  # Compute the covariance matrix.
  covariance_matrix = np.cov(data.T)

  # Compute the eigenvalues and eigenvectors of the covariance matrix.
  eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

  # Sort the eigenvalues and eigenvectors in descending order by eigenvalue.
  eigenvalues, eigenvectors = eigenvalues[::-1], eigenvectors[:, ::-1]

  # Choose the first `n_components` eigenvectors and eigenvalues.
  eigenvectors = eigenvectors[:, :n_components]
  eigenvalues = eigenvalues[:n_components]

  # Transform the data into the principal component space.
  transformed_data = np.dot(data, eigenvectors)

  return eigenvectors, eigenvalues, transformed_data


In [12]:
eigenvectors, eigenvalues, transformed_data = pca(df.drop(columns="label"), 400)

In [13]:
data = pd.DataFrame(transformed_data)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,1.713907e-15-2.896988e-16j,1.713907e-15+2.896988e-16j,8.604228e-16+0.000000e+00j,-7.771561e-16+0.000000e+00j,1.908196e-15-1.496199e-16j,1.908196e-15+1.496199e-16j,5.689893e-16+7.008283e-16j,5.689893e-16-7.008283e-16j,-4.302114e-16+1.068590e-15j,-4.302114e-16-1.068590e-15j,...,-1.151856e-15-4.510281e-16j,-1.151856e-15+4.510281e-16j,-1.477984e-15+7.147061e-16j,-1.477984e-15-7.147061e-16j,-1.561251e-17+1.672273e-15j,-1.561251e-17-1.672273e-15j,-1.804112e-16-7.771561e-16j,-1.804112e-16+7.771561e-16j,6.748074e-16+1.526557e-16j,6.748074e-16-1.526557e-16j
1,5.273559e-16+2.116363e-16j,5.273559e-16-2.116363e-16j,-4.128642e-16+0.000000e+00j,2.003606e-16+0.000000e+00j,8.881784e-16+4.163336e-17j,8.881784e-16-4.163336e-17j,1.665335e-16+2.359224e-16j,1.665335e-16-2.359224e-16j,-5.273559e-16-2.836273e-16j,-5.273559e-16+2.836273e-16j,...,-7.494005e-16-5.134781e-16j,-7.494005e-16+5.134781e-16j,-8.326673e-17+1.387779e-16j,-8.326673e-17-1.387779e-16j,-2.775558e-17+9.540979e-17j,-2.775558e-17-9.540979e-17j,-1.554312e-15-8.118506e-16j,-1.554312e-15+8.118506e-16j,-5.204170e-16+7.771561e-16j,-5.204170e-16-7.771561e-16j
2,1.387779e-15+2.584738e-16j,1.387779e-15-2.584738e-16j,-9.992007e-16+0.000000e+00j,-7.771561e-16+0.000000e+00j,2.317591e-15-3.816392e-17j,2.317591e-15+3.816392e-17j,-8.326673e-17+2.636780e-16j,-8.326673e-17-2.636780e-16j,-1.125836e-15+2.220446e-16j,-1.125836e-15-2.220446e-16j,...,-2.359224e-16-5.967449e-16j,-2.359224e-16+5.967449e-16j,8.049117e-16+1.894318e-15j,8.049117e-16-1.894318e-15j,-2.192690e-15+1.193490e-15j,-2.192690e-15-1.193490e-15j,-1.665335e-16-3.226586e-16j,-1.665335e-16+3.226586e-16j,-8.743006e-16+2.636780e-16j,-8.743006e-16-2.636780e-16j
3,2.154527e-15+3.122502e-16j,2.154527e-15-3.122502e-16j,-7.216450e-16+0.000000e+00j,6.106227e-16+0.000000e+00j,2.213507e-15+4.510281e-17j,2.213507e-15-4.510281e-17j,1.179612e-15+9.506285e-16j,1.179612e-15-9.506285e-16j,-1.290634e-15+2.498002e-16j,-1.290634e-15-2.498002e-16j,...,-6.245005e-16-4.544976e-16j,-6.245005e-16+4.544976e-16j,-7.702172e-16+1.062735e-15j,-7.702172e-16-1.062735e-15j,-4.302114e-16+4.718448e-16j,-4.302114e-16-4.718448e-16j,-6.106227e-16-4.440892e-16j,-6.106227e-16+4.440892e-16j,-1.365227e-15-7.285839e-16j,-1.365227e-15+7.285839e-16j
4,1.026956e-15+1.543904e-16j,1.026956e-15-1.543904e-16j,-5.828671e-16+0.000000e+00j,-3.330669e-16+0.000000e+00j,1.464107e-15+1.040834e-17j,1.464107e-15-1.040834e-17j,8.326673e-17-4.145989e-16j,8.326673e-17+4.145989e-16j,-7.216450e-16-1.387779e-17j,-7.216450e-16+1.387779e-17j,...,-1.165734e-15-4.163336e-16j,-1.165734e-15+4.163336e-16j,8.604228e-16+2.303713e-15j,8.604228e-16-2.303713e-15j,-1.762479e-15+1.013079e-15j,-1.762479e-15-1.013079e-15j,-3.747003e-16-5.828671e-16j,-3.747003e-16+5.828671e-16j,-5.551115e-17+1.533496e-15j,-5.551115e-17-1.533496e-15j


In [15]:
data["labels"] = df["label"]
data.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,labels
0,1.713907e-15-2.896988e-16j,1.713907e-15+2.896988e-16j,8.604228e-16+0.000000e+00j,-7.771561e-16+0.000000e+00j,1.908196e-15-1.496199e-16j,1.908196e-15+1.496199e-16j,5.689893e-16+7.008283e-16j,5.689893e-16-7.008283e-16j,-4.302114e-16+1.068590e-15j,-4.302114e-16-1.068590e-15j,...,-1.151856e-15+4.510281e-16j,-1.477984e-15+7.147061e-16j,-1.477984e-15-7.147061e-16j,-1.561251e-17+1.672273e-15j,-1.561251e-17-1.672273e-15j,-1.804112e-16-7.771561e-16j,-1.804112e-16+7.771561e-16j,6.748074e-16+1.526557e-16j,6.748074e-16-1.526557e-16j,alaa
1,5.273559e-16+2.116363e-16j,5.273559e-16-2.116363e-16j,-4.128642e-16+0.000000e+00j,2.003606e-16+0.000000e+00j,8.881784e-16+4.163336e-17j,8.881784e-16-4.163336e-17j,1.665335e-16+2.359224e-16j,1.665335e-16-2.359224e-16j,-5.273559e-16-2.836273e-16j,-5.273559e-16+2.836273e-16j,...,-7.494005e-16+5.134781e-16j,-8.326673e-17+1.387779e-16j,-8.326673e-17-1.387779e-16j,-2.775558e-17+9.540979e-17j,-2.775558e-17-9.540979e-17j,-1.554312e-15-8.118506e-16j,-1.554312e-15+8.118506e-16j,-5.204170e-16+7.771561e-16j,-5.204170e-16-7.771561e-16j,alaa
2,1.387779e-15+2.584738e-16j,1.387779e-15-2.584738e-16j,-9.992007e-16+0.000000e+00j,-7.771561e-16+0.000000e+00j,2.317591e-15-3.816392e-17j,2.317591e-15+3.816392e-17j,-8.326673e-17+2.636780e-16j,-8.326673e-17-2.636780e-16j,-1.125836e-15+2.220446e-16j,-1.125836e-15-2.220446e-16j,...,-2.359224e-16+5.967449e-16j,8.049117e-16+1.894318e-15j,8.049117e-16-1.894318e-15j,-2.192690e-15+1.193490e-15j,-2.192690e-15-1.193490e-15j,-1.665335e-16-3.226586e-16j,-1.665335e-16+3.226586e-16j,-8.743006e-16+2.636780e-16j,-8.743006e-16-2.636780e-16j,alaa
3,2.154527e-15+3.122502e-16j,2.154527e-15-3.122502e-16j,-7.216450e-16+0.000000e+00j,6.106227e-16+0.000000e+00j,2.213507e-15+4.510281e-17j,2.213507e-15-4.510281e-17j,1.179612e-15+9.506285e-16j,1.179612e-15-9.506285e-16j,-1.290634e-15+2.498002e-16j,-1.290634e-15-2.498002e-16j,...,-6.245005e-16+4.544976e-16j,-7.702172e-16+1.062735e-15j,-7.702172e-16-1.062735e-15j,-4.302114e-16+4.718448e-16j,-4.302114e-16-4.718448e-16j,-6.106227e-16-4.440892e-16j,-6.106227e-16+4.440892e-16j,-1.365227e-15-7.285839e-16j,-1.365227e-15+7.285839e-16j,alaa
4,1.026956e-15+1.543904e-16j,1.026956e-15-1.543904e-16j,-5.828671e-16+0.000000e+00j,-3.330669e-16+0.000000e+00j,1.464107e-15+1.040834e-17j,1.464107e-15-1.040834e-17j,8.326673e-17-4.145989e-16j,8.326673e-17+4.145989e-16j,-7.216450e-16-1.387779e-17j,-7.216450e-16+1.387779e-17j,...,-1.165734e-15+4.163336e-16j,8.604228e-16+2.303713e-15j,8.604228e-16-2.303713e-15j,-1.762479e-15+1.013079e-15j,-1.762479e-15-1.013079e-15j,-3.747003e-16-5.828671e-16j,-3.747003e-16+5.828671e-16j,-5.551115e-17+1.533496e-15j,-5.551115e-17-1.533496e-15j,alaa


In [16]:
X_train, X_test, y_train, t_test = train_test_split(data.drop(columns="labels"),data["labels"], test_size=0.15, random_state=42)

In [18]:
params = {
    "kernel" : ["linear","poly","rbf"],
    'C':[0.01, 0.1, 0.5, 5, 10],
    'gamma': [1,0.1,0.01,0.001]
}
model = GridSearchCV(estimator=SVC(), param_grid=params, n_jobs=-1, cv=5, verbose=1)
model.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


ValueError: 
All the 300 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\DELL\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 856, in check_array
    array = np.asarray(array, order=order, dtype=dtype)
  File "c:\Users\DELL\anaconda3\lib\site-packages\pandas\core\generic.py", line 2064, in __array__
    return np.asarray(self._values, dtype=dtype)
numpy.ComplexWarning: Casting complex values to real discards the imaginary part

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\DELL\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_tr